In [1]:
import pandas as pd
import requests
import json
import getpass
import re
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import random
import re

In [11]:
rapidAPIkey = getpass.getpass()

········


In [3]:
#1 INFORMATION ON AVAILABLE WORLDWIDE ROUTES

#A Establishing a dataset to have all available worldwide routes
#https://www.kaggle.com/open-flights/flight-route-database
routes = pd.read_csv('routes.csv')
routes = routes.rename(columns={'airline ID':'airline_id',' source airport':'source_airport',
                                ' source airport id':'source_airport_id',
                                ' destination apirport':'destination_airport',
                                ' destination airport id':'destination_airport_id',' codeshare':'codeshare',
                                ' stops':'stops',' equipment':'equipment'})

routes = routes.drop(['airline','airline_id','codeshare','stops','equipment'],axis=1)

#BCITY CODES AND CITY NAMES PER ROUTE

#The routes dataset gives as IATA codes but not cities
#Thus, we will scrape wikipedia to get the city names

#list of all letters in the alpabet:
A=ord('A')
alph=[chr(i) for i in range(A,A+26)]

#Scrapping wikipedia to get airport codes and city
cities = []
codes = []
for letter in alph:
    url = f'https://en.wikipedia.org/wiki/List_of_airports_by_IATA_airport_code:_{letter}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    soup_cities = soup.select('td:nth-child(4)')
    soup_codes = soup.select("td:nth-child(1)")
    for i in soup_cities:
        cities.append(i.get_text())
    for i in soup_codes:
        codes.append(i.get_text())
        
#creating a dataframe with wikipedia information
airports_cities = pd.DataFrame({"city":cities,"airport_code":codes})

#cleaning the dataset created
def clean_n(x):
    return re.sub('\n','',x)

airports_cities['city'] = airports_cities['city'].apply(clean_n)

#adding a column with only the city
airports_cities['real_city'] = airports_cities['city'].str.split(',').str[0]



#correcting the city_column with /:
airports_cities['real_city'] = airports_cities['real_city'].str.split('/').str[0]

#deleting parenthesis od city_column:
airports_cities['real_city'] = airports_cities['real_city'].str.split('(').str[0]

#C FINAL DATASET

#Merging wikipedia with the routes dataset (origin)
routes_origin = pd.merge(left = routes,
                                 right = airports_cities,
                                 how = 'left', 
                                 left_on = "source_airport", 
                                 right_on= "airport_code")
routes_origin = routes_origin.rename(columns={'real_city':'origin_city','city':'origin_place',
                                              'source_airport':'origin_airport'})
routes_origin = routes_origin.drop(['source_airport_id','destination_airport_id','airport_code'],axis=1)


#Merging routes origin with wikipedia to get the cities of the destination
routes_final = pd.merge(left = routes_origin,
                                 right = airports_cities,
                                 how = 'left', 
                                 left_on = "destination_airport", 
                                 right_on= "airport_code")
routes_final = routes_final.rename(columns={'city':'destination_place','source_place':'origin_place',
                                            'airport_code':'destination_airport2','real_city':'destination_city',
                                           })
#droping unecessary columns
routes_final = routes_final.drop(['destination_airport2'],axis=1)

#reordering columns
routes_final = routes_final[['origin_airport','origin_city','origin_place',
              'destination_airport','destination_city','destination_place']]

#Creation of an excel with the 35 missing airports to fill them in: (origin_city)
missing_airports = pd.read_excel('missing_airports.xlsx')

routes_final = pd.merge(left = routes_final,
                                 right = missing_airports,
                                 how = 'left', 
                                 left_on = "origin_airport", 
                                 right_on= "IATA_code")

routes_final['origin_city'] = routes_final['origin_city'].fillna(routes_final['City'])
routes_final = routes_final.drop(['IATA_code','City'],axis=1)

#Trying the same for the destination:

routes_final = pd.merge(left = routes_final,
                                 right = missing_airports,
                                 how = 'left', 
                                 left_on = "destination_airport", 
                                 right_on= "IATA_code")

routes_final['destination_city'] = routes_final['destination_city'].fillna(routes_final['IATA_code'])
routes_final = routes_final.drop(['IATA_code','City'],axis=1)

In [4]:
#2 CUSTOMER INPUTS

#Asking the customer hor many origins will there be:

while True:
    input_user = input("Amount of origins: ")
    try:
        amount_of_origins = int(input_user)
        if amount_of_origins < 0:  
            print("Sorry, input must be a positive integer, try again")
            continue
        break
    except ValueError:
        print("That's not an int!")  

#Asking the customer which origins are?
origins_list = []
for i in range(amount_of_origins):
    origins_list.append(input(f'Origin {i+1} ',))

#Asking the customer the dates
departure_date = input('Departure date in format YYYY-MM-DD: ')
return_date = input('Return date in format YYYY-MM-DD: ')

Amount of origins: 3
Origin 1 Barcelona
Origin 2 Madrid
Origin 3 Valencia
Departure date in format YYYY-MM-DD: 2021-05-31
Return date in format YYYY-MM-DD: 2021-06-06


In [5]:
#3 COMMON DESTINATIONS (origin -> destination)

#For the origins inputed, we have to get the common destinations:

#First we create a dictionary with all destinations for each origin:
origins_dict = {}
for i in range(len(origins_list)):
    origins_dict["%s" %origins_list[i]] = list(routes_final['destination_city'][routes_final['origin_city']==origins_list[i]].unique())

#We convert all destinations in a list of lists:
values_list = list(origins_dict.values())

#We convert the list of lists into a simple list:
def flatten(input_list):
    return [item for sublist in input_list for item in sublist]

values_list = flatten(values_list)

#We convert the list into a dataframe
destinations = pd.DataFrame(values_list)
destinations.columns = ['all_destinations']

#We choose only the destinations that appear as many times as the amount of origins we have
destinations2 = pd.DataFrame(destinations['all_destinations'].value_counts())
common_destinations = destinations2[destinations2['all_destinations']>=len(origins_list)]

destinations_both = common_destinations.index.tolist()

#ALERT BOX:
if len(origins_list) != amount_of_origins:
    print('Problem with origins!!')
elif len(destinations_both) == 0:
    print('Problem with destinations!!')
else:
    print("All good, let's continue")
    print('Origins: ',origins_list)
    print('Total common destinations: ',len(destinations_both))

All good, let's continue
Origins:  ['Barcelona', 'Madrid', 'Valencia']
Total common destinations:  46


In [13]:
#4 LISTS OF CITY CODES (ORIGIN AND DESTINATION)

#City code function definition:
def city_code(city_name):
    url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/US/USD/en-US/"
    params = {"query": city_name}
    headers = {'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",'x-rapidapi-key': rapidAPIkey}
    response = requests.get(url, headers = headers, params = params)
    if len(response.json()["Places"]) == 0:
        return response.json()["Places"]
    else:
        return response.json()["Places"][0]["PlaceId"]

#First we create a list with the city code all origins 
origins_codes = []
for i in origins_list:
    origins_codes.append(city_code(i))
    wait_time = random.uniform(1.2, 1.2)
    print("Looking for the code of: " + i + ' city. It will take me: ' + str(round(wait_time,2)) + ' seconds.')
    sleep(wait_time)

origin_codes_vf = []
for i in origins_codes:
    if i != []:
        origin_codes_vf.append(i)

#Now we create a list with the city code all destinations 
destinations_both_codes = []
for i in destinations_both:
    destinations_both_codes.append(city_code(i))
    wait_time = random.uniform(1.2, 1.2)
    print("Looking for the code of: " + i + ' city. It will take me: ' + str(round(wait_time,2)) + ' seconds.')
    sleep(wait_time)

#Now we delete the destinations that are not found in sky scanner
destinations_both_codes_vf = []
for i in destinations_both_codes:
    if i != []:
        destinations_both_codes_vf.append(i)

Looking for the code of: Barcelona city. It will take me: 1.2 seconds.
Looking for the code of: Madrid city. It will take me: 1.2 seconds.
Looking for the code of: Valencia city. It will take me: 1.2 seconds.
Looking for the code of: Charleroi city. It will take me: 1.2 seconds.
Looking for the code of: Sofia city. It will take me: 1.2 seconds.
Looking for the code of: Bristol city. It will take me: 1.2 seconds.
Looking for the code of: Dublin city. It will take me: 1.2 seconds.
Looking for the code of: Timișoara city. It will take me: 1.2 seconds.
Looking for the code of: Santander city. It will take me: 1.2 seconds.
Looking for the code of: Marseille city. It will take me: 1.2 seconds.
Looking for the code of: Istanbul city. It will take me: 1.2 seconds.
Looking for the code of: Panama City city. It will take me: 1.2 seconds.
Looking for the code of: Cluj-Napoca city. It will take me: 1.2 seconds.
Looking for the code of: Beauvais city. It will take me: 1.2 seconds.
Looking for the c

In [14]:
#5 MINIMUM PRICES (EUROS)

#now we defind a function to get the minimum prices:
def minimum_prices(origin, destination, date):
    url = f"https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browseroutes/v1.0/ES/EUR/en-EUR/{origin}/{destination}/{date}"
    #querystring = {"inboundpartialdate":{date}}
    headers = {'x-rapidapi-key': rapidAPIkey,'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com"}
    response = requests.request("GET", url, headers=headers)#, params=querystring)
    if response.json()['Quotes'] == []:
        my_dict = {}
        return my_dict
    else:
        quotes = pd.json_normalize(response.json()['Quotes'])
        carriers = pd.json_normalize(response.json()['Carriers'])
        data = pd.concat([quotes,carriers],axis=1).reset_index()
        my_dict = {'origin':origin,'destination':destination,'date':date,'min_price':min(data['MinPrice'])}  
        return my_dict

#now we create a list with all the minimum prices:
new_list = []
for i in destinations_both_codes_vf:
    for o in origin_codes_vf:
        new_list.append(minimum_prices(o, i, departure_date))
        wait_time = random.uniform(1.20, 1.20)
        print("Looking for the minimum price of a flight from: " + o + ' to: ' + i + '. It will take me: ' + str(round(wait_time,2)) + " seconds.")
        sleep(wait_time)
          
#We now store everything in a dataframe:
minimum_prices_df = pd.DataFrame(new_list)

#Deleting nulls:
minimum_prices_df = minimum_prices_df[minimum_prices_df['origin'].isna()==False]
minimum_prices_df = minimum_prices_df.reset_index()
minimum_prices_df = minimum_prices_df.drop(['index'],axis=1)

#Deleting destinations without flights on the days specified to one or more cities of origin
minimum_prices_destinations = pd.DataFrame(minimum_prices_df['destination'].value_counts())
minimum_prices_destinations = minimum_prices_destinations[minimum_prices_destinations['destination']>=len(origins_list)]

#MINIMUM PRICES of the return:
#data_return1 = data.rename(columns={'origin':'destination_code','destination':'origin_code'}) #revisar si es pot eliminar!
#data_return1 = data_return1.drop(['date','min_price'],axis=1)
#
#return_destination_list = list(set(data_return1['destination_code'].to_list()))
return_destination_list = list(minimum_prices_df['origin'].unique())

return_origin_list = minimum_prices_destinations.index.to_list()
#now we create a list with all the minimum prices:
new_list_return = []
for i in return_destination_list:
    for o in return_origin_list:
        new_list_return.append(minimum_prices(o, i, return_date))
        wait_time = random.uniform(1.20, 1.20000001)
        print("Looking for the minimum price of a flight from: " + o + ' to: ' + i + '. It will take me: ' + str(round(wait_time,2)) + " seconds.")
        sleep(wait_time)

#Delete the destinations that are not found in sky scanner for the return date
new_list_return_vf = []
for i in new_list_return:
    if i != {}:
        new_list_return_vf.append(i)

data_return = pd.DataFrame(new_list_return_vf)

#renaming dataframe of return
data_return = data_return.rename(columns={'origin':'return_origin','destination':'return_destination',
                                          'min_price':'return_min_price','date':'return_date'})

Looking for the minimum price of a flight from: BCN-sky to: CRL-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: MAD-sky to: CRL-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: VLC-sky to: CRL-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: BCN-sky to: SOF-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: MAD-sky to: SOF-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: VLC-sky to: SOF-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: BCN-sky to: BRS-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: MAD-sky to: BRS-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: VLC-sky to: BRS-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: BCN-sky to: DUB-sky. It will take me: 1.2 seconds.
Looking fo

Looking for the minimum price of a flight from: VLC-sky to: MOSC-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: BCN-sky to: LIS-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: MAD-sky to: LIS-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: VLC-sky to: LIS-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: BCN-sky to: GVA-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: MAD-sky to: GVA-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: VLC-sky to: GVA-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: BCN-sky to: PRG-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: MAD-sky to: PRG-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: VLC-sky to: PRG-sky. It will take me: 1.2 seconds.
Looking f

Looking for the minimum price of a flight from: LIS-sky to: BCN-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: LPA-sky to: BCN-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: FRAN-sky to: BCN-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: NTE-sky to: BCN-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: BRUS-sky to: BCN-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: MILA-sky to: BCN-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: AGP-sky to: BCN-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: BLQ-sky to: BCN-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: VIE-sky to: BCN-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: CMN-sky to: MAD-sky. It will take me: 1.2 seconds.
Looking

Looking for the minimum price of a flight from: NTE-sky to: VLC-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: BRUS-sky to: VLC-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: MILA-sky to: VLC-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: AGP-sky to: VLC-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: BLQ-sky to: VLC-sky. It will take me: 1.2 seconds.
Looking for the minimum price of a flight from: VIE-sky to: VLC-sky. It will take me: 1.2 seconds.


In [15]:
#8 FINAL DATAFRAME

#DATA DEPARTURE

#Dataframe with all destinations names
df_city_names = {'city_name':destinations_both,'city_code':destinations_both_codes}
df_city_names = pd.DataFrame(df_city_names)
#function to do an index match
def xlookup(lookup_value, lookup_array, return_array, if_not_found:str = ''):
    match_value = return_array.loc[lookup_array == lookup_value]
    if match_value.empty:
        return f'"{lookup_value}" not found!' if if_not_found == '' else if_not_found
    else:
        return match_value.tolist()[0]
#Adding the city name of the destination to the dataset:
minimum_prices_df['destination_city'] = minimum_prices_df['destination'].apply(xlookup, args = (df_city_names['city_code'], df_city_names['city_name']))
#Repeating the process for the origins:
origin_names = {'city_name':origins_list,'city_code':origins_codes}
origin_names = pd.DataFrame(origin_names) #Revisar que no estigui en funció de l'ordre!

#Adding the origins codes:
minimum_prices_df['origin_city'] = minimum_prices_df['origin'].apply(xlookup, args = (origin_names['city_code'], origin_names['city_name']))

#DATA RETURN

new_df = pd.merge(minimum_prices_df, data_return,  how='left', left_on=['origin','destination'], right_on = ['return_destination','return_origin'])
new_df['price_departure_and_return'] = new_df['min_price'] + new_df['return_min_price']

#Deleting nulls (null = no return flight)
new_df = new_df[new_df['return_origin'].isna()==False]
new_df = new_df.reset_index()
new_df = new_df.drop(['index'],axis=1)

#TOTAL PRICE TO PAY

#Compute the total price for each destination (Summing what to pay for each Destination for each origin)
#Example:
# BCN -> LISBOA 25
# MAD -> LISBOA 75
# list 2 = [100]

check_return_flights = pd.DataFrame(new_df['destination_city'].value_counts())
check_return_flights = check_return_flights[check_return_flights['destination_city']>=len(origins_list)]
destination_list = list(set(check_return_flights.index.to_list()))

list2 = []
for i in destination_list:
    list2.append(i)
    list2.append(new_df['price_departure_and_return'][new_df['destination_city']==i].sum())

#Puting the result into a dataframe

#dividing the list into two list
values_list = []
keys_list = []
for i in range(len(list2)):
    if i % 2 == 0:
        keys_list.append(list2[i])
    else:
        values_list.append(list2[i])

#creating a dictionary
res = {keys_list[i]: values_list[i] for i in range(len(keys_list))}

#converting the dictionary into a dataframe
df_final = pd.DataFrame(values_list,keys_list).reset_index()
df_final.columns = ['destination','total_price'] #revisar pq es poden girar les columnes i ja no funciona!!!
df_final = df_final.sort_values(by=['total_price'])
df_final = df_final.reset_index()
df_final = df_final.drop(['index'],axis=1)
#Final destination:

#Creating a dataframe for final destination and price
final_destination_and_price = df_final[df_final['total_price']>0]
final_destination_and_price = final_destination_and_price[final_destination_and_price['total_price']==min(final_destination_and_price['total_price'])]
final_destination_and_price = final_destination_and_price.reset_index()
final_destination_and_price = final_destination_and_price.drop(['index'], axis=1)

#creating variables to store them:
final_destination = final_destination_and_price['destination'][0]
final_price = final_destination_and_price['total_price'][0]

In [16]:
#THE RESULT!

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#FINAL DECISION:
print('Origins: ',origins_list)
print('From:',departure_date)
print('To:',return_date)

print('')
print('You should meet in: ',final_destination)
print('For a total price of: ',int(final_price),'€')
print('Price per person: ',int(final_price/len(origins_list)),'€')
print('')
print('See the prices of each flight:')
data_to_show = new_df[new_df['destination_city']==final_destination]
data_to_show = data_to_show.loc[:,['origin_city','date','min_price','destination_city','return_date','return_min_price','price_departure_and_return']]
data_to_show = data_to_show.reset_index()
data_to_show = data_to_show.drop(['index'],axis=1)
data_to_show = data_to_show.rename(columns={'date':'departure_date','origin_city':'departure_city',
                             'min_price':'departure_flight_price',
                            'return_min_price':'return_flight_price',
                             'price_departure_and_return':'price_departure_and_return'})
data_to_show
print('')
print('See other options:')
df_final = df_final[df_final['total_price']>0]
df_final = df_final.sort_values(by=['total_price']).reset_index()
df_final = df_final.drop(['index'],axis=1)
df_final['price_per_person'] = round(df_final['total_price']/len(origins_list),2)
#df_final.loc[[1,2,3],:]
df_final.head()

Origins:  ['Barcelona', 'Madrid', 'Valencia']
From: 2021-05-31
To: 2021-06-06

You should meet in:  Porto
For a total price of:  158 €
Price per person:  52 €

See the prices of each flight:

See other options:


,destination,total_price,price_per_person
0,Porto,158.0,52.67
1,Dublin,188.0,62.67
2,Rome,190.0,63.33
3,Seville,198.0,66.00
4,Bologna,202.0,67.33


In [13]:
#HOTEL SEARCHING!


In [17]:
#Define the destinations in which we will search for a hotel: (the best options according to flights)
final_hf_df = df_final.head()
final_hf_df = final_hf_df.rename(columns={'total_price':'flights_total_price','price_per_person':'flight_price_per_person'})
hotels_cities_list = final_hf_df['destination'].to_list()


#Response for every city:
response_different_citites = []
for i in hotels_cities_list:
    url1 = "https://hotels4.p.rapidapi.com/locations/search"
    querystring1 = {"query":i}
    headers1 = {'x-rapidapi-key': rapidAPIkey ,'x-rapidapi-host': "hotels4.p.rapidapi.com"}
    response_different_citites.append(requests.request("GET", url1, headers=headers1, params=querystring1))

#Response for every city json:
response_different_citites_json = []
for i in range(len(response_different_citites)):
    response_different_citites_json.append(response_different_citites[i].json())

#Getting the destination ids for all cities
all_destination_ids = []

for i in range(len(response_different_citites_json)):
    for o in range(len(response_different_citites_json[i]['suggestions'])):
        for w in range(len(response_different_citites_json[i]['suggestions'][o]['entities'])):
            all_destination_ids.append(response_different_citites_json[i]['suggestions'][o]['entities'][w]['destinationId'])

response_list = []
for i in range(len(all_destination_ids)):
    url2 = "https://hotels4.p.rapidapi.com/properties/list"
    querystring2 = {"adults1":len(origins_list),"pageNumber":"1","destinationId":all_destination_ids[i],"pageSize":"25","checkOut":return_date,"checkIn":departure_date,"sortOrder":"PRICE","currency":"EUR"}
    headers2 = {'x-rapidapi-key': rapidAPIkey,
            'x-rapidapi-host': "hotels4.p.rapidapi.com"}
    response_list.append(requests.request("GET", url2, headers=headers2, params=querystring2))
    
response_json = []
for i in range(len(response_list)):
    response_json.append(response_list[i].json())

In [18]:
#CREATING A DATA FRAME:

#Address:
city_list = []
address_list = []
for i in range(len(response_json)):
    if 'data' in response_json[i].keys():
        for o in range(len(response_json[i]['data']['body']['searchResults']['results'])):
            city_list.append(response_json[i]['data']['body']['searchResults']['results'][o]['address']['locality'])
            if 'streetAddress' in response_json[i]['data']['body']['searchResults']['results'][o]['address'].keys():
                address_list.append(response_json[i]['data']['body']['searchResults']['results'][o]['address']['streetAddress'])
    else:
        address_list.append('Adress not found')
        city_list.append('Adress not found')

adress_df = pd.concat([pd.DataFrame(address_list),pd.DataFrame(city_list)],axis=1)
adress_df.columns = ['address','city']

#Extracting prices:
price_per_night_list = []
price_per_stay_list = []
for i in range(len(response_json)):
    if 'data' in response_json[i].keys():
        for o in range(len(response_json[i]['data']['body']['searchResults']['results'])):
            price_per_night_list.append(response_json[i]['data']['body']['searchResults']['results'][o]['ratePlan']['price']['exactCurrent'])
            price_per_stay_list.append(response_json[i]['data']['body']['searchResults']['results'][o]['ratePlan']['price']['totalPricePerStay'])
    
price_df = pd.concat([pd.DataFrame(price_per_night_list),pd.DataFrame(price_per_stay_list)],axis=1)    
price_df.columns = ['price_per_night_and_person','total_price_per_stay_and_person']
#price_df['price_per_night_and_person'] = price_df['price_per_night_and_person'].str.split(' ').str[0]    
price_df['price_per_night_and_person'] = pd.to_numeric(price_df['price_per_night_and_person'],downcast='float')

#Filtering for the distance to the city center:
def price_stay_clean(x):
    x = str(x)
    x = x.split('(<strong>')
    x = x[1]
    x = x.split('</strong>')
    x = x[0]
    x = x.split(' ')
    x = x[0]
    x = x.replace('.', '')
    x = float(x)
    return x
price_df['total_price_per_stay_and_person'] = price_df['total_price_per_stay_and_person'].apply(price_stay_clean)

distance_center_list = []
for i in range(len(response_json)):
    if 'data' in response_json[i].keys():
        for o in range(len(response_json[i]['data']['body']['searchResults']['results'])):
            if len(response_json[i]['data']['body']['searchResults']['results'][o]['landmarks']) > 0:
                distance_center_list.append(response_json[i]['data']['body']['searchResults']['results'][o]['landmarks'][0]['distance'])
            else:
                distance_center_list.append(None)
    else:
        distance_center_list.append(None)


distance_df = pd.DataFrame(distance_center_list)
distance_df.columns = ['distance_to_city_center']

distance_df['distance_to_city_center_in_km'] = distance_df['distance_to_city_center'].str.split(' ').str[0]    
distance_df["distance_to_city_center_in_km"] = pd.to_numeric(distance_df["distance_to_city_center_in_km"], downcast="float")
distance_df["distance_to_city_center_in_km"] = round(distance_df["distance_to_city_center_in_km"]*1.60934,1)
distance_df = distance_df.drop(['distance_to_city_center'],axis=1)


#Filtering to have a rating higher than X
rating_list = []
rating_scale = []
for i in range(len(response_json)):
    if 'data' in response_json[i].keys():
        for o in range(len(response_json[i]['data']['body']['searchResults']['results'])):
            if 'guestReviews' in (response_json[i]['data']['body']['searchResults']['results'][o].keys()):
                rating_list.append(response_json[i]['data']['body']['searchResults']['results'][o]['guestReviews']['rating'])
                rating_scale.append(response_json[i]['data']['body']['searchResults']['results'][o]['guestReviews']['scale'])
            else:
                rating_list.append(0)
                rating_scale.append(0)

rating_df = pd.concat([pd.DataFrame(rating_list),pd.DataFrame(rating_scale)],axis=1)
rating_df.columns = ['rating','rating_scale']

rating_df["rating"] = pd.to_numeric(rating_df["rating"], downcast="float")
rating_df['rating_scale_10'] = rating_df['rating']/rating_df['rating_scale']*10
#rating_df['rating_scale_10'] = rating_df['rating_scale_10'] .fillna('No reviews found')
rating_df = rating_df.drop(['rating','rating_scale'],axis=1)


#hotel name:
name_list = []
for i in range(len(response_json)):
    if 'data' in response_json[i].keys():
        for o in range(len(response_json[i]['data']['body']['searchResults']['results'])):
            name_list.append(response_json[i]['data']['body']['searchResults']['results'][o]['name'])

name_df = pd.DataFrame(name_list)
name_df.columns = ['hotel_name']

hotels_df = pd.concat([name_df,adress_df,price_df,distance_df,rating_df],axis=1)
hotels_df = pd.DataFrame(hotels_df)
hotels_df = hotels_df[hotels_df['hotel_name'].isna()==False]
hotels_df = hotels_df[hotels_df['price_per_night_and_person'].isna()==False]
hotels_df = hotels_df.reset_index()
hotels_df = hotels_df.drop(['index'],axis=1)

hotels_df['check_in_date'] = departure_date
hotels_df['check_out_date'] = return_date

miami_list = []
for i in pd.DataFrame(hotels_df['city'].value_counts()).index.to_list():
    if re.findall('Miami',i) == ['Miami']:
        miami_list.append(i)
hotels_df['city'].value_counts()

hotels_df[hotels_df['city']=='London']

Rome                      224
Dublin                    196
Bologna                   183
Seville                   138
Porto                      40
                         ... 
Carro                       1
Borzonasca                  1
Castiglione Chiavarese      1
Voghiera                    1
Clio                        1
Name: city, Length: 67, dtype: int64

,hotel_name,address,city,price_per_night_and_person,total_price_per_stay_and_person,distance_to_city_center_in_km,rating_scale_10,check_in_date,check_out_date


In [19]:
#THE RESULT: V3
rating_input = input('Minimum rating of the hotel (scale of 10): ')
distance_input = input('Maximum distance to the center (in km): ')
print('Destination: ',final_destination)
print('Departure date: ',departure_date)
print('Return date: ',return_date)
print('')
print('With this conditions, MeetPlace recommends:')
hotels_df = hotels_df[hotels_df['distance_to_city_center_in_km']<=float(distance_input)]
hotels_df = hotels_df[hotels_df['rating_scale_10']>=float(rating_input)]
hotels_df = hotels_df.sort_values(by=['total_price_per_stay_and_person'])

meetplace_v3 = hotels_df[hotels_df['city']==final_destination]
meetplace_v3 = meetplace_v3.head(10)
meetplace_v3 = meetplace_v3.reset_index()
meetplace_v3 = meetplace_v3.drop(['index'],axis=1)
meetplace_v3 = meetplace_v3.loc[:,['hotel_name','address','price_per_night_and_person','total_price_per_stay_and_person','distance_to_city_center_in_km','rating_scale_10']]
meetplace_v3

Minimum rating of the hotel (scale of 10): 7
Maximum distance to the center (in km): 3
Destination:  Porto
Departure date:  2021-05-31
Return date:  2021-06-06

With this conditions, MeetPlace recommends:


,hotel_name,address,price_per_night_and_person,total_price_per_stay_and_person,distance_to_city_center_in_km,rating_scale_10
0,Residencial Vale Formoso,"Rua da Alegria, 313",33.389999,200.0,1.3,7.6
1,D&S - Porto São Bento Apartment II,Rua do Heroísmo,34.680000,208.0,1.1,8.6
2,Porto Island,"Rua de Santa Teresa, 19 1 Esq",36.169998,217.0,2.6,9.0
3,RS Porto Flats,Rua da Alegria 642,40.889999,245.0,0.6,7.8
4,Trindade Modern Flat,"Rua Doutor Alves da Veiga, 58",41.139999,247.0,0.3,8.0
5,Studios Porto Solar,"Via del Fondaco, 11",41.509998,249.0,0.6,8.2
6,Alves da Veiga Apartments,"Rua da Assembleia, 193",42.459999,255.0,1.0,8.6
7,FMO Apartments Boavista,"Praca 10, s/n",42.450001,255.0,1.6,8.8
8,Pedra Ibérica,Salita Baratta 16,42.450001,255.0,1.0,8.8
9,Lapa 32,"Praça Dezessete, 33",42.459999,255.0,0.3,7.6


In [20]:
hotels_df_destination1 =hotels_df[hotels_df['city']==hotels_cities_list[0]].head(25)
hotels_df_destination2 =hotels_df[hotels_df['city']==hotels_cities_list[1]].head(25)
hotels_df_destination3 =hotels_df[hotels_df['city']==hotels_cities_list[2]].head(25)
hotels_df_destination4 =hotels_df[hotels_df['city']==hotels_cities_list[3]].head(25)
hotels_df_destination5 =hotels_df[hotels_df['city']==hotels_cities_list[4]].head(25)

import numpy as np
mean_destination1 = round(np.mean(hotels_df_destination1['total_price_per_stay_and_person']),2)
mean_destination2  = round(np.mean(hotels_df_destination2['total_price_per_stay_and_person']),2)
mean_destination3 = round(np.mean(hotels_df_destination3['total_price_per_stay_and_person']),2)
mean_destination4 = round(np.mean(hotels_df_destination4['total_price_per_stay_and_person']),2)
mean_destination5 = round(np.mean(hotels_df_destination5['total_price_per_stay_and_person']),2)


average_hotel_price_list = [mean_destination1,mean_destination2,mean_destination3,mean_destination4,mean_destination5]

mean_hotel_price_df = pd.concat([pd.DataFrame(hotels_cities_list),pd.DataFrame(average_hotel_price_list)],axis=1)
mean_hotel_price_df.columns = ['city','avg_price_per_stay_and_person']

print('From: ',origins_list)
print('The cheapest place to meet is: ',final_destination)
#print('Destination: ',final_destination)
print('Departure date: ',departure_date)
print('Return date: ',return_date)
print('Minimum rating of hotels',rating_input)
print('Maximum distance to city center',distance_input)
#print('Total expenditure (flight + average hotel price per stay): ','784.55','€')
final_solution = pd.merge(left = final_hf_df,
                                 right = mean_hotel_price_df,
                                 how = 'left', 
                                 left_on = "destination", 
                                 right_on= "city")
final_solution = final_solution.drop(['flights_total_price','city'],axis=1)
final_solution['total_expenditure'] = final_solution['flight_price_per_person'] + final_solution['avg_price_per_stay_and_person']
final_solution
print('')
print('See the prices of each flight:')
data_to_show
print('')

print('')
print('With these conditions, MeetPlace recommends you the following hotels:')
hotels_df = hotels_df[hotels_df['distance_to_city_center_in_km']<=float(distance_input)]
hotels_df = hotels_df[hotels_df['rating_scale_10']>=float(rating_input)]
meetplace_v3

From:  ['Barcelona', 'Madrid', 'Valencia']
The cheapest place to meet is:  Porto
Departure date:  2021-05-31
Return date:  2021-06-06
Minimum rating of hotels 7
Maximum distance to city center 3


,destination,flight_price_per_person,avg_price_per_stay_and_person,total_expenditure
0,Porto,52.67,238.60,291.27
1,Dublin,62.67,450.16,512.83
2,Rome,63.33,203.92,267.25
3,Seville,66.00,273.96,339.96
4,Bologna,67.33,355.16,422.49



See the prices of each flight:


,departure_city,departure_date,departure_flight_price,destination_city,return_date,return_flight_price,price_departure_and_return
0,Barcelona,2021-05-31,45.0,Porto,2021-06-06,18.0,63.0
1,Madrid,2021-05-31,32.0,Porto,2021-06-06,14.0,46.0
2,Valencia,2021-05-31,33.0,Porto,2021-06-06,16.0,49.0




With these conditions, MeetPlace recommends you the following hotels:


,hotel_name,address,price_per_night_and_person,total_price_per_stay_and_person,distance_to_city_center_in_km,rating_scale_10
0,Residencial Vale Formoso,"Rua da Alegria, 313",33.389999,200.0,1.3,7.6
1,D&S - Porto São Bento Apartment II,Rua do Heroísmo,34.680000,208.0,1.1,8.6
2,Porto Island,"Rua de Santa Teresa, 19 1 Esq",36.169998,217.0,2.6,9.0
3,RS Porto Flats,Rua da Alegria 642,40.889999,245.0,0.6,7.8
4,Trindade Modern Flat,"Rua Doutor Alves da Veiga, 58",41.139999,247.0,0.3,8.0
5,Studios Porto Solar,"Via del Fondaco, 11",41.509998,249.0,0.6,8.2
6,Alves da Veiga Apartments,"Rua da Assembleia, 193",42.459999,255.0,1.0,8.6
7,FMO Apartments Boavista,"Praca 10, s/n",42.450001,255.0,1.6,8.8
8,Pedra Ibérica,Salita Baratta 16,42.450001,255.0,1.0,8.8
9,Lapa 32,"Praça Dezessete, 33",42.459999,255.0,0.3,7.6
